### 目标
* 通过代码生成训练的图片和标签
* 将自己的训练集合喂给PaddlePaddle

In [ ]:
import numpy
numpy.random.random(size=(1,13)).astype('float32')
#numpy.random.random(size=(1,1)).astype('float32')
#print(a)
#print(b)

## numpy知识准备

In [ ]:
import numpy as np
numpy.array([[1.1,2.2,3.3]],dtype='float32')

#a = 4
#b = [[a]]

aa=np.empty([0,13],'float32')
bb = [1.1,2.2,3.2,4.3,5,6,7,8,9,10,11,12,13]
cc=[bb] 
aa = np.append(aa, cc,axis=0)
print(aa)

## 训练数据准备
图片按照需要存入
```
./img/num.png
```
记录文件 trainData.dt
```
./img/num.png,0
./img/num.png,1
```
新建img目录

* [ ] 代码生成目录

In [ ]:
import numpy as np
import cv2 as cv
import random

imgW = 28
imgH = 28

def generalRec():
    img = np.ones((imgW,imgH),dtype=np.uint8)
    InitX = random.randint(2,imgW-5)
    InitY = random.randint(2,imgH-5)
    RecW = random.randint(4,imgW-InitX-1)
    RecH = random.randint(4,imgH-InitY-1)
    cv.rectangle(img,(InitX,InitY),(RecW,RecH),255,1)
#     #显示图片
#     cv.namedWindow("img")
#     cv.imshow("img",img)
#     cv.waitKey(0)
#     cv.destroyAllWindows()
    return img


    
def generalCircle():
    img = np.ones((imgW,imgH),dtype=np.uint8)
    InitX = random.randint(5,imgW-5)
    InitY = random.randint(5,imgH-5)
    minDis = min(5,imgW-InitX,imgH-InitY)
    r = random.randint(2,minDis)
    cv.circle(img,(InitX,InitY),r,255,1)
#     #显示图片
#     cv.namedWindow("img")
#     cv.imshow("img",img)
#     cv.waitKey(0)
#     cv.destroyAllWindows()
    return img
    

def GenerateTrainData(genNum):
    with open('trainData.dt','w') as f:
        for i in range(genNum):
            randBoo = random.choice([True,False])
            if(randBoo):
                # general rec
                genImg = generalRec()
                lineStr = "./img/"+i.__str__()+".png,0\n"
                f.write(lineStr)
                cv.imwrite("./img/"+i.__str__()+".png",genImg)
            else:
                # general circle
                genImg = generalCircle()
                lineStr = "./img/"+i.__str__()+".png,1\n"
                f.write(lineStr)
                cv.imwrite("./img/"+i.__str__()+".png",genImg)
            
        

GenerateTrainData(10)

## 读取测试

In [17]:
import numpy as np
import cv2 as cv




with open('trainData.dt') as f:
    line = f.readline()
    while line:
        strSplit=line.split(',',1)
        label_input=float(strSplit[1])
        aa=numpy.empty([0,1],'float32')
        aa = numpy.append(aa, [[label_input]],axis=0)
        print(aa)
        
#        img = cv.imread(strSplit[0],cv.IMREAD_GRAYSCALE)
#         npImg = np.reshape(img,(1,784))
#         print(npImg)
#         cv.namedWindow("img")
#         cv.imshow("img",img)
#         cv.waitKey(0)
#         cv.destroyAllWindows()
        line = f.readline()


[[0.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]


## PaddlePadlle

In [19]:
import paddle
import paddle.fluid as fluid
import numpy
import math
import sys
import cv2 as cv

# For training test cost
def train_test(executor, program, reader, feeder, fetch_list):
    accumulated = 1 * [0]
    count = 0
    for data_test in reader():
        outs = executor.run(
            program=program, feed=feeder.feed(data_test), fetch_list=fetch_list)
        accumulated = [x_c[0] + x_c[1][0] for x_c in zip(accumulated, outs)]
        count += 1
    return [x_d / count for x_d in accumulated]


def main():
    batch_size = 20
    train_reader = paddle.batch(
        paddle.reader.shuffle(paddle.dataset.uci_housing.train(), buf_size=500),
        batch_size=batch_size)
    test_reader = paddle.batch(
        paddle.reader.shuffle(paddle.dataset.uci_housing.test(), buf_size=500),
        batch_size=batch_size)

    # feature vector of length 784
    x = fluid.layers.data(name='x', shape=[784], dtype='float32')
    y = fluid.layers.data(name='y', shape=[1], dtype='float32')
    y_predict = fluid.layers.fc(input=x, size=1, act=None)

    main_program = fluid.default_main_program()
    startup_program = fluid.default_startup_program()

    cost = fluid.layers.square_error_cost(input=y_predict, label=y)
    avg_loss = fluid.layers.mean(cost)

    sgd_optimizer = fluid.optimizer.SGD(learning_rate=0.001)
    sgd_optimizer.minimize(avg_loss)

    test_program = main_program.clone(for_test=True)

    # can use CPU or GPU
    use_cuda = True
    place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
    exe = fluid.Executor(place)

    # Specify the directory to save the parameters
    params_dirname = "fit_a_line.inference.model"
    num_epochs = 100

    # main train loop.
    feeder = fluid.DataFeeder(place=place, feed_list=[x, y])
    exe.run(startup_program)

    train_prompt = "Train cost"
    test_prompt = "Test cost"
    step = 0

    exe_test = fluid.Executor(place)

#     aa=numpy.empty([0,784],'float32')
#     bb = [1.1,2.2,3.2,4.3,5,6,7,8,9,10,11,12,13]
#     cc=[bb] 
#     aa = numpy.append(aa, cc,axis=0)
    
    
    with open('trainData.dt') as f:
        line = f.readline()
        while line:
            strSplit=line.split(',',1)
            img = cv.imread(strSplit[0],cv.IMREAD_GRAYSCALE)
            npImg = numpy.reshape(img,(1,784))
            label_input=float(strSplit[1])
            npLabel=numpy.empty([0,1],'float32')
            npLabel = numpy.append(npLabel, [[label_input]],axis=0)
            avg_loss_value, = exe.run(
                main_program,
                feed={
                       "x": npImg.astype('float32'),
                       "y": npLabel.astype('float32')
                },
                fetch_list=[avg_loss])
            step += 1
            if step % 1 == 0:  # record a train cost every 10 batches
                print("%s, Step %d, Cost %f" %
                      (train_prompt, step, avg_loss_value[0]))
            line = f.readline()

#            if step % 100 == 0:  # record a test cost every 100 batches
#                test_metics = train_test(
#                    executor=exe_test,
#                    program=test_program,
#                    reader=test_reader,
#                    fetch_list=[avg_loss],
#                    feeder=feeder)
#                print("%s, Step %d, Cost %f" %
#                      (test_prompt, step, test_metics[0]))
                # If the accuracy is good enough, we can stop the training.
#                if test_metics[0] < 10.0:
#                    break
            

#             if math.isnan(float(avg_loss_value[0])):
#                 sys.exit("got NaN loss, training failed.")
#         if params_dirname is not None:
#             # We can save the trained parameters for the inferences later
#             fluid.io.save_inference_model(params_dirname, ['x'], [y_predict],
#                                           exe)

#     infer_exe = fluid.Executor(place)
#     inference_scope = fluid.core.Scope()

    # infer
#  
#     with fluid.scope_guard(inference_scope):
#         [inference_program, feed_target_names, fetch_targets
#          ] = fluid.io.load_inference_model(params_dirname, infer_exe)
#         batch_size = 10

#         infer_reader = paddle.batch(
#             paddle.dataset.uci_housing.test(), batch_size=batch_size)

#         infer_data = next(infer_reader())
#         infer_feat = numpy.array(
#             [data[0] for data in infer_data]).astype("float32")
#         infer_label = numpy.array(
#             [data[1] for data in infer_data]).astype("float32")

#         assert feed_target_names[0] == 'x'
#         results = infer_exe.run(
#             inference_program,
#             feed={feed_target_names[0]: numpy.array(infer_feat)},
#             fetch_list=fetch_targets)

#         print("infer results: (House Price)")
#         for idx, val in enumerate(results[0]):
#             print("%d: %.2f" % (idx, val))

#         print("\nground truth:")
#         for idx, val in enumerate(infer_label):
#             print("%d: %.2f" % (idx, val))
 

if __name__ == '__main__':
    main()


Train cost, Step 1, Cost 860.028503
Train cost, Step 2, Cost 73699528.000000
Train cost, Step 3, Cost 5575144898560.000000
Train cost, Step 4, Cost 3612168159232000.000000
Train cost, Step 5, Cost 1181186236605712564224.000000
Train cost, Step 6, Cost 113870718149340934390677504.000000
Train cost, Step 7, Cost 10082978098649916212403077382144.000000
Train cost, Step 8, Cost 14609593939467829587911910898008064.000000
Train cost, Step 9, Cost 3281871344954872226710874796526141440.000000
Train cost, Step 10, Cost inf


## 小结
* 自己生产的数据可以通过opencv和np喂给Paddle
* 训练效果有问题 
    * 分类和回归问题的目标函数和损失函数不一样 以二维为例
        * 回归是为了找到拟合数据的曲线
        * 分类是为了找到数据的分割线
        
 Todo：
 * [ ] 使用FC、SoftMax网络做分类
 * [ ] 数据读取改用自定义Reader的方式